In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!nvidia-smi

Thu Jan 20 16:31:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel

     |████████████████████████████████| 792 kB 28.8 MB/s 
     |████████████████████████████████| 374 kB 71.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.24 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.0 which is incompatible.
     |████████████████████████████████| 127 kB 29.4 MB/s 
ERROR: Operation cancelled by user


In [1]:
cd /content/drive/MyDrive/Human-Path-Prediction-master (1)

/content/drive/MyDrive/Human-Path-Prediction-master (1)


In [2]:
cd ynet

/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet


In [3]:
!ls

 config				   requirements.txt
 data				   segmentation_models
'evaluate_inD_longterm(1).ipynb'   test.py
 evaluate_inD_longterm.ipynb	   train_inD_longterm.ipynb
 evaluate_SDD_longterm.ipynb	   train.py
 evaluate_SDD_trajnet.ipynb	   train_SDD_longterm.ipynb
 images				   train_SDD_trajnet.ipynb
 model.py			   utils
 pretrained_models		   w_andb
 __pycache__			   weights_and_biases.py
 README.md


In [7]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 9.6 MB/s 
     |████████████████████████████████| 97 kB 7.3 MB/s 
     |████████████████████████████████| 142 kB 54.7 MB/s 
     |████████████████████████████████| 180 kB 53.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=caf67b892832cce9a6676226533ad2d4787c4abfd68c5242685e7114af748948
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=335fac3d88b13d5a8032c8476ea6449aadc79f7efcba0d6f9b572edeb43b238f
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [4]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet
import weights_and_biases as wandb

In [5]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [7]:
CONFIG_FILE_PATH = 'config/sdd_trajnet.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'sdd_trajnet'  # arbitrary name for this experiment
DATASET_NAME = 'sdd'

TRAIN_DATA_PATH = 'data/SDD/train_trajnet.pkl'
TRAIN_IMAGE_PATH = 'data/SDD/train'
VAL_DATA_PATH = 'data/SDD/test_trajnet.pkl'
VAL_IMAGE_PATH = 'data/SDD/test'
OBS_LEN = 8  # in timesteps
PRED_LEN = 12  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

BATCH_SIZE = 2

#### Load config file and print hyperparameters

In [11]:
pip install -r requirements.txt

     |████████████████████████████████| 67 kB 3.5 MB/s 
     |████████████████████████████████| 269 kB 14.9 MB/s 
     |████████████████████████████████| 753.2 MB 14 kB/s 
     |████████████████████████████████| 49.4 MB 1.2 MB/s 
     |████████████████████████████████| 25.9 MB 1.3 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
     |████████████████████████████████| 23.3 MB 647 kB/s 
     |████████████████████████████████| 23.3 MB 75.4 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 336 kB/s 
     |████████████████████████████████| 17.4 MB 89 kB/s 
     |████████████████████████████████| 17.3 MB 50.2 MB/s 
     |████████████████████████████████| 12.8 MB 56.0 MB/s 
     |████████████████████████████████| 12.7 MB 46.2 MB/s 
     |████████████████████████████████| 11.8 MB 61.5 MB/s 
     |████████████████████████████████| 5.9 MB 60.9 MB/s 
     |██████████

In [8]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': 'None',
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 300,
 'rel_threshold': 0.01,
 'resize': 0.25,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.0,
 'unfreeze': 150,
 'use_CWS': False,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [11]}

#### Load preprocessed Data

In [9]:
!pip3 install pickle5
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)

import pickle5 as pickle 
with open(TRAIN_DATA_PATH, "rb") as fh:
    df_train = pickle.load(fh)
with open(VAL_DATA_PATH, "rb") as fh1:
    df_val = pickle.load(fh1)

     |████████████████████████████████| 256 kB 8.5 MB/s 


In [ ]:
import pickle5 as pickle 
with open("/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet/data/eth_ucy/df_final_train.pkl", "rb") as fh:
    df_train1 = pickle.load(fh)
with open("/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet/data/eth_ucy/df_final_test.pkl", "rb") as fh1:
    df_val1 = pickle.load(fh1)

In [ ]:
#print("Count of students with physics marks equal is->",
#      df_train1[df_train1['trackId'] ==34]['frame'].count())

In [10]:
#import sys
#sys.setrecursionlimit(100001)
df_train.head()

,frame,trackId,x,y,sceneId,metaId
0,0,100,757.0,918.5,bookstore_0,0
1,12,100,765.0,918.5,bookstore_0,0
2,24,100,773.0,918.5,bookstore_0,0
3,36,100,780.5,918.5,bookstore_0,0
4,48,100,788.5,919.5,bookstore_0,0


In [11]:
print("Count of students with physics marks greater than 11 is->",
      df_train[df_train['trackId'] == 35]['frame'].count())

Count of students with physics marks greater than 11 is-> 480


In [ ]:
print(VAL_DATA_PATH)

/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet/data/eth_ucy/df_final_test.pkl


In [ ]:
print(type(df_val))
print(type(df_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


#### Initiate model and load pretrained weights

In [12]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

In [ ]:
!wandb login --relogin
wandb.init_wandb(params.copy(), model.model)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: agv (use `wandb login --relogin` to force relogin)


In [ ]:
wandb.init_wandb(params.copy(), model.model)

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [ ]:
#sys. setrecursionlimit(100000001)
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH,
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|██████████| 240/240 [00:00<00:00, 861.38it/s]
/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet/utils/dataloader.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(trajectories), meta, scene_list
Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

Start training
240


RuntimeError: ignored